<a href="https://colab.research.google.com/github/GenesiaW/Google-Colab-Mangadex-Reader/blob/main/MangaDex_Reader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title MangaDex Reader
#@markdown After running, click the 3 dots on right and select 'view output fullscreen'
import requests
import json
import ipywidgets as widgets
from IPython.display import display
from math import ceil
from urllib import parse
from IPython.display import clear_output
from traitlets import traitlets
from google.colab import files
from zipfile import ZipFile

class LoadedButton(widgets.Button):
    """A button that can holds a value as a attribute."""

    def __init__(self, value=None, *args, **kwargs):
        super(LoadedButton, self).__init__(*args, **kwargs)
        # Create the value attribute.
        self.add_traits(value=traitlets.Any(value))

class LoadedTab(widgets.Tab):
    """A Tab that can holds a value as a attribute."""

    def __init__(self, value=None, *args, **kwargs):
        super(LoadedTab, self).__init__(*args, **kwargs)
        # Create the value attribute.
        self.add_traits(value=traitlets.Any(value))


def search(query,offset=0):
    encoded = parse.quote(query)
    if doujin_filter.value:
      manga_obj = {
      "title":query,
      "limit":25,
      "offset":offset,
    }
    else:
      manga_obj = {
      "title":query,
      "limit":25,
      "offset":offset,
      "excludedTags[]":"b13b2a48-c720-44a9-9c77-39c9979373fb",
    }
    response = requests.get('https://api.mangadex.org/manga', params=manga_obj)
    parsedjson = json.loads(response.text)
    total_pages = ceil(parsedjson['total']/25)
    result_list = parsedjson['results']
    title_list = {i['data']['attributes']['title']['en']:i['data']['id'] for i in result_list}
    manga_list = [i['data']['attributes']['title']['en'] for i in result_list]
    print(f'======= Manga List ({offset//25 +1}/{total_pages}) =======')
    for i in range(len(result_list)):
      print(f'{offset+i+1}. ' + result_list[i]['data']['attributes']['title']['en'])
    return manga_list,title_list,parsedjson['total'],offset
def chapterlist(manga_id,offset=0):
    chapter_obj = {
      "limit":25,
      "offset":offset,
      "manga":manga_id,
      "translatedLanguage":"en",
      "order[publishAt]":'desc',
    }
    response = requests.get('https://api.mangadex.org/chapter',params=chapter_obj)
    parsedjson = json.loads(response.text)
    total_pages = ceil(parsedjson['total']/25)
    result_list = parsedjson['results']
    chapter_list = [i['data']['id'] for i in result_list]
    chapter_titles = []
    print(f'======= Chapter List ({offset//25 +1}/{total_pages}) =======')
    for i in range(len(result_list)):
        chapter_titles.append(f'{offset+i+1}. Chapter '+result_list[i]['data']['attributes']['chapter'] + ' '
              + result_list[i]['data']['attributes']['title'])
        print(f'{offset+i+1}. Chapter '+ result_list[i]['data']['attributes']['chapter'] + ' '
              + result_list[i]['data']['attributes']['title'])
    return chapter_list,parsedjson['total'],offset,chapter_titles

def reader(chapter_id):
  response = requests.get('https://api.mangadex.org/chapter/' + chapter_id)
  parsedjson = json.loads(response.text)
  data = parsedjson['data']
  chapterhash = data['attributes']['hash']
  images = data['attributes']['data']
  base_url = json.loads(requests.get('https://api.mangadex.org/at-home/server/' + chapter_id).text)['baseUrl']
  total_pages = len(images)
  image_list = [base_url + '/data/' + chapterhash +'/' + images[i] for i in range(len(images))]
  return image_list

search_box = widgets.Text(
    value='',
    placeholder='Input Manga Title here',
    disabled=False,
)
confirm_search = LoadedButton(
    description = 'Search',
    value=0,
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
)

doujin_filter = widgets.Checkbox(
    value = False,
    description='Doujinshi',
    disabled=False
)
output = widgets.Output()
output2 = widgets.Output()
output3 = widgets.Output()
search_bar = widgets.HBox([search_box,confirm_search,doujin_filter])

mode_list = widgets.Dropdown(
    options=['Page','List'],#[i+1 for i in range(len(y[0]))],
    value='Page',
    disabled=False,
)

def return_search_box_val(b):
  b.button_style='success'
  with output:
    clear_output()
    confirm_search.value = search(search_box.value)
    b.button_style=''
    print('')
    manga_list.options = [i+1 for i in range(len(confirm_search.value[0]))]
    if confirm_search.value[3] > 0:
      left_page.disabled=False
      left_page.value=confirm_search.value[3]-25
    else:
      left_page.disabled=True
    manga_list.disabled = False
    confirm_manga.disabled=False
    if confirm_search.value[2]-confirm_search.value[3] < 26:
      right_page.disabled=True
    else:
      if confirm_search.value[2] <25:
        right_page.disabled=True
      else:
        right_page.disabled=False
        right_page.value=confirm_search.value[3]+25
left_page = LoadedButton(
  description='<',
  value=0,
  disabled=True #False if y[3] > 0 else True,
)
manga_list = widgets.Dropdown(
    options=[1,2],#[i+1 for i in range(len(y[0]))],
    value=1,
    disabled=True,
)
confirm_manga= LoadedButton(
    description='Select',
    value=0,
    disabled=True
)
right_page = LoadedButton(
    description='>',
    value=0,
    disabled=True #True if y[2]-y[3] < 0 else False,
)

#Tab 1 Buttons

left_page_c = LoadedButton(
  description='<',
  value=0,
  disabled=True
)
chapters_list = widgets.Dropdown(
    options=[1,2],
    value=1,
    disabled=False,
)
confirm_chapter= LoadedButton(
    description='Select',
    value=0,
    disabled=False
)
right_page_c = LoadedButton(
    description='>',
    disabled=True,
    value=0
)

skip_to_front_c = LoadedButton(
    description='|<--',
    disabled=True,
    value=0
)

skip_to_back_c = LoadedButton(
    description='-->|',
    disabled=True,
    value=0
)

#Tab 3 Butons:
data = LoadedButton(
    description='Jump',
    value=0,
    disabled=False 
)

page_slider = widgets.Dropdown(
    options=[1,2],
    layout=widgets.Layout(width='5%')
)

left_page_r = LoadedButton(
  description='<',
  value=0,
  disabled=True
)

right_page_r = LoadedButton(
    description='>',
    value=0,
    disabled=True
)

def on_right_page_click(b):
  b.button_style='success'
  with output:
    clear_output()
    confirm_search.value = search(search_box.value,right_page.value)
    b.button_style=''
    print('')
    manga_list.options = [right_page.value+i+1 for i in range(len(confirm_search.value[0]))]
    if confirm_search.value[3] > 0:
      left_page.disabled=False
      left_page.value=confirm_search.value[3]-25
    else:
      left_page.disabled=True
    if confirm_search.value[2]-confirm_search.value[3] < 26:
      right_page.disabled=True
    else:
      if confirm_search.value[2] <25:
        right_page.disabled=True
      else:
        right_page.disabled=False
        right_page.value=confirm_search.value[3]+25

def on_left_page_click(b):
  b.button_style='success'
  with output:
    clear_output()
    confirm_search.value = search(search_box.value,left_page.value)
    b.button_style=''
    print('')
    manga_list.options = [left_page.value+i+1 for i in range(len(confirm_search.value[0]))]
    if confirm_search.value[3] > 0:
      left_page.disabled=False
      left_page.value=confirm_search.value[3]-25
    else:
      left_page.disabled=True
    if confirm_search.value[2]-confirm_search.value[3] < 26:
      right_page.disabled=True
    else:
      if confirm_search.value[2] <25:
        right_page.disabled=True
      else:
        right_page.disabled=False
        right_page.value=confirm_search.value[3]+25

def on_confirm_manga_click(b):
  b.button_style='success'
  if manga_list.value%25:
    confirm_manga.value = confirm_search.value[1][confirm_search.value[0][(manga_list.value%25)-1]]
    download.value = confirm_search.value[0][(manga_list.value%25)-1]
  else:
    confirm_manga.value = confirm_search.value[1][confirm_search.value[0][24]]
    download.value = confirm_search.value[0][24]
  b.button_style=''
  tab.selected_index=1
  with output2:
    clear_output()
    tab.value = chapterlist(confirm_manga.value)
    print('')
  skip_to_front_c.value=0
  skip_to_front_c.disabled=False
  if tab.value[2] > 0:
    left_page_c.value=tab.value[2]-25,
    left_page_c.disabled=False
  else:
    left_page_c.value=0,
    left_page_c.disabled=True
  if tab.value[1]-tab.value[2]<26:
    right_page_c.disabled=True
    right_page_c.value=0
  else:
    if tab.value[1] <25:
      right_page_c.disabled=True
      skip_to_back_c.disabled=True
    else:
      skip_to_back_c.value=tab.value[1]-25
      skip_to_back_c.disabled=False
      right_page_c.disabled=False
      right_page_c.value=tab.value[2]+25
      
  chapters_list.options=[i for i in tab.value[3]]

confirmation = widgets.HBox([left_page,manga_list,confirm_manga,right_page])
container_1 = widgets.VBox([search_bar,output,confirmation])

#Tab 0 Button Functions
confirm_search.on_click(return_search_box_val)
right_page.on_click(on_right_page_click)
left_page.on_click(on_left_page_click)
confirm_manga.on_click(on_confirm_manga_click)

#Tab 1 Button Functions
def on_right_page_c_click(b):
  b.button_style='success'
  with output2:
    clear_output()
    tab.value  = chapterlist(confirm_manga.value,right_page_c.value)
    b.button_style=''
    print('')
    chapters_list.options = [i for i in tab.value[3]]
    if tab.value[2] > 0:
      left_page_c.disabled=False
      left_page_c.value=tab.value[2]-25
    else:
      left_page_c.disabled=True
    if tab.value[1]-tab.value[2] < 26:
      right_page_c.disabled=True
    else:
      if tab.value[1] <25:
        right_page_c.disabled=True
        skip_to_back_c.disabled=True
      else:
        skip_to_back_c.value=tab.value[1]-25
        skip_to_back_c.disabled=False
        right_page_c.disabled=False
        right_page_c.value=tab.value[2]+25

def on_left_page_c_click(b):
  b.button_style='success'
  with output2:
    clear_output()
    y = chapterlist(confirm_manga.value,left_page_c.value)
    tab.value = y
    b.button_style=''
    print('')
    chapters_list.options = [i for i in tab.value[3]]
    if y[2] > 0:
      left_page_c.disabled=False
      left_page_c.value=y[2]-25
    else:
      left_page_c.disabled=True
    if y[1]-y[2] < 26:
      right_page_c.disabled=True
    else:
      if tab.value[1] <25:
        right_page_c.disabled=True
        skip_to_back_c.disabled=True
      else:
        skip_to_back_c.value=tab.value[1]-25
        skip_to_back_c.disabled=False
        right_page_c.disabled=False
        right_page_c.value=tab.value[2]+25

def on_skip_to_front_c_click(b):
  b.button_style='success'
  with output2:
    clear_output()
    y = chapterlist(confirm_manga.value,skip_to_front_c.value)
    tab.value = y
    b.button_style=''
    print('')
    chapters_list.options = [i for i in tab.value[3]]
    if y[2] > 0:
      left_page_c.disabled=False
      left_page_c.value=y[2]-25
    else:
      left_page_c.disabled=True
    if y[1]-y[2] < 26:
      right_page_c.disabled=True
    else:
      if tab.value[1] <25:
        right_page_c.disabled=True
        skip_to_back_c.disabled=True
      else:
        skip_to_back_c.value=tab.value[1]-25
        skip_to_back_c.disabled=False
        right_page_c.disabled=False
        right_page_c.value=tab.value[2]+25

def on_skip_to_back_c_click(b):
  b.button_style='success'
  with output2:
    clear_output()
    tab.value  = chapterlist(confirm_manga.value,skip_to_back_c.value)
    skip_to_back_c.disabled=True
    b.button_style=''
    print('')
    chapters_list.options = [i for i in tab.value[3]]
    if tab.value[2] > 0:
      left_page_c.disabled=False
      left_page_c.value=tab.value[2]-25
    else:
      left_page_c.disabled=True
    if tab.value[1]-tab.value[2] < 26:
      right_page_c.disabled=True
    else:
      if tab.value[1] <25:
        right_page_c.disabled=True
        skip_to_back_c.disabled=True
      else:
        skip_to_back_c.value=tab.value[1]-25
        skip_to_back_c.disabled=False
        right_page_c.disabled=False
        right_page_c.value=tab.value[2]+25

def on_confirm_chapter_click(b):
  output4.clear_output()
  with output4:
    output3.clear_output()
  left_page_r.value=0
  right_page_r.value=0
  left_page_r.disabled=True
  b.button_style='success'
  if type(download.value) == tuple:
    download.value = download.value[0],chapters_list.value
  else:
    download.value = download.value,chapters_list.value
  index_id = tab.value[3].index(chapters_list.value)
  confirm_chapter.value=tab.value[0][index_id]
  b.button_style=''
  tab.selected_index=2
  url_lists = reader(confirm_chapter.value)
  image_widgets = [widgets.HTML(value=f'<img src="{i}">')for i in url_lists]
  total_pages_r = len(image_widgets)
  page_slider.options=[i+1 for i in range(total_pages_r)]
  page_slider.value=page_slider.options[0]
  data.value = url_lists,image_widgets,total_pages_r
  if mode_list.value == "List":
    data.disabled=True
    page_slider.disabled=True
    right_page_r.disabled=True
    left_page_r.disabled=True
    for i in data.value[1]:
      with output4:
        output3.append_display_data(i)
    with output4:
      display(output3)
  elif mode_list.value == "Page":
    data.disabled=False
    page_slider.disabled=False
    with output4:
      output3.append_display_data(data.value[1][0]) 
      display(output3)
    if right_page_r.value < data.value[2]:
      right_page_r.value += 1
      right_page_r.disabled=False

#Reader Functions

def on_right_page_r_click(b):
  output4.clear_output()
  with output4:
    output3.clear_output()
  b.button_style='success'
  with output4:
    output3.append_display_data(data.value[1][right_page_r.value])
  with output4:
    display(output3)
  b.button_style=''
  left_page_r.value = right_page_r.value -1
  left_page_r.disabled=False
  right_page_r.value += 1
  page_slider.value = right_page_r.value
  if right_page_r.value < data.value[2]:
    right_page_r.disabled=False
  elif right_page_r.value == data.value[2]:
    right_page_r.disabled=True
  if left_page_r.value < 0:
    left_page_r.disabled=True

def on_left_page_r_click(b):
  b.button_style='success'
  with output4:
    clear_output()
  output4.clear_output()
  with output4:
    output3.clear_output()
  with output4:
    output3.append_display_data(data.value[1][left_page_r.value])
  with output4:
    display(output3)
  b.button_style=''
  right_page_r.value = left_page_r.value +1
  page_slider.value = right_page_r.value
  left_page_r.value -= 1
  if right_page_r.value < data.value[2]:
    right_page_r.disabled=False
  elif right_page_r.value == data.value[2]:
    right_page_r.disabled=True
  if left_page_r.value < 0:
    left_page_r.disabled=True

def on_data_click(b):
  b.button_style='success'
  with output4:
    output4.clear_output()
  output4.clear_output()
  with output4:
    output3.append_display_data(data.value[1][page_slider.value-1])
  with output4:
    display(output3)
  b.button_style=''
  right_page_r.value = page_slider.value
  left_page_r.value = page_slider.value - 2
  if left_page_r.value < page_slider.value:
    left_page_r.disabled=False
  if right_page_r.value < data.value[2]:
    right_page_r.disabled=False
  elif right_page_r.value == data.value[2]:
    right_page_r.disabled=True
  if left_page_r.value < 0:
    left_page_r.disabled=True

download=LoadedButton(
  description="Download",
  value=0,
)

def on_download_click(b):
  b.button_style="success"
  output4.clear_output()
  with output4:
    output3.clear_output()
  directory_name = "'" + download.value[1] + "'"
  zipped_file_name = "'" + download.value[0]+'.zip' + "'"
  !mkdir $directory_name
  with ZipFile(download.value[0]+'.zip','w') as ZipObj:
    for i in range(len(data.value[0])):
      img_data = requests.get(data.value[0][i]).content
      page_name =f'page{i+1}.jpg'
      with open(f'{download.value[1]}/{page_name}', 'wb') as handler:
        handler.write(img_data)
      ZipObj.write(f'{download.value[1]}/{page_name}')
      !rm -rf $page_name
  files.download(download.value[0]+'.zip')
  b.button_style=""


confirm_chapter.on_click(on_confirm_chapter_click)
right_page_c.on_click(on_right_page_c_click)
left_page_c.on_click(on_left_page_c_click)
skip_to_back_c.on_click(on_skip_to_back_c_click)
skip_to_front_c.on_click(on_skip_to_front_c_click)

right_page_r.on_click(on_right_page_r_click)
left_page_r.on_click(on_left_page_r_click)
data.on_click(on_data_click)
download.on_click(on_download_click)

confirmation_c = widgets.HBox([skip_to_front_c,left_page_c,chapters_list,confirm_chapter,right_page_c,skip_to_back_c])
container_2 = widgets.VBox([output2,confirmation_c])

output4 = widgets.Output()

pages_control = widgets.HBox([left_page_r,chapters_list,confirm_chapter,right_page_r,page_slider,data,download])
container_3 = widgets.VBox([pages_control,output4,pages_control])


tab=LoadedTab()
tab.children = [container_1,container_2,container_3,mode_list]
tab.set_title(0,'Search For Manga')
tab.set_title(1,'Chapters List')
tab.set_title(2,'Reader')
tab.set_title(3,'Reader Settings')
tab

In [ ]:
#@title Run if download manga was used
!rm -rf /content/*